In [0]:
from google.colab import files
uploaded = files.upload()

Saving test.data to test.data
Saving test.label to test.label
Saving train.data to train.data
Saving train.label to train.label


In [0]:
label_path

'train.label'

In [0]:
import csv 
import math
import numpy
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

print('starting to train..')


label_path = 'train.label'
data_path = 'train.data'
number_of_class=20;
number_of_words=61188
num_of_doc=11269

# construct label dict
f = open(label_path)
reader = csv.reader(f,delimiter=" ")
label=numpy.zeros(shape=(num_of_doc,1))
i=0;
for rows in reader:
    label[i]=int(rows[0])-1
    i+=1
# calculate df=number of docs containing word w 
df=numpy.zeros(shape=(number_of_words,1))
# clacualte word freq per doc
X=numpy.zeros(shape=(num_of_doc,number_of_words))

f = open(data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    X[int(rows[0])-1][int(rows[1])-1]=int(rows[2]) 
    df[int(rows[1])-1]+=1
print("calculated x and df")




starting to train..
calculated x and df


In [0]:
from sklearn.decomposition import TruncatedSVD

# clac tf-idf
tfidf=numpy.zeros(shape=(num_of_doc,number_of_words))
for d in range(0,num_of_doc):
  if d%1000==0:
    print("doc count: "+str(d))
  for w in range(0,number_of_words):
    if df[w]==0:
      tfidf[d][w]=0
    else:
      tfidf[d][w]=X[d][w]*math.log(num_of_doc/df[w])
print("calculated tfidf")


doc count: 0
doc count: 1000
doc count: 2000
doc count: 3000
doc count: 4000
doc count: 5000
doc count: 6000
doc count: 7000
doc count: 8000
doc count: 9000
doc count: 10000
doc count: 11000
calculated tfidf
starting do svd
finished


In [0]:
print("starting do svd")
#  reduce dimension of vector using svd
svd = TruncatedSVD(n_components=5000)
train_data = svd.fit_transform(tfidf)

print("finished")

starting do svd
finished


In [0]:
del X
del df
del tfidf
print("cleared memory")

cleared memory


In [41]:
# "using sklearn to test the result"
print("started..")
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
clf.fit(train_data,label)
print("model fitting comleted")


started..


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


model fitting comleted


In [43]:
print("testing")
prediction=clf.predict(test_data)
print("prediction completed")

correct=0
incorrect=0
for i in range(0,no_of_test_doc):
    if prediction[i]==tlabel[i]:
        correct+=1
    else:
        incorrect+=1

print("final result: c: "+str(correct)+"  e: "+str(incorrect))
print("Accuracy:"+str(correct/(correct+incorrect)))


testing
prediction completed
final result: c: 5057  e: 2448
Accuracy:0.67381745502998


In [0]:
print('starting to buid x and df for test')
test_label_path = 'test.label'
test_data_path = 'test.data'
no_of_test_doc=7505


# calculate df=number of docs containing word w 
df_test=numpy.zeros(shape=(number_of_words,1))
# clacualte word freq per doc
X_test=numpy.zeros(shape=(no_of_test_doc,number_of_words))

f = open(test_data_path)
reader = csv.reader(f,delimiter=" ")
for rows in reader:
    X_test[int(rows[0])-1][int(rows[1])-1]=int(rows[2]) 
    df_test[int(rows[1])-1]+=1

print("calculated x and df")


starting to buid x and df for test
calculated x and df


In [0]:
# clac tf-idf
tfidf_test=numpy.zeros(shape=(no_of_test_doc,number_of_words))
for d in range(0,no_of_test_doc):
    if d%1000==0:
      print('count:'+str(d))
    for w in range(0,number_of_words):
      if df_test[w]==0:
        tfidf_test[d][w]=0
      else:
        tfidf_test[d][w]=X_test[d][w]*math.log(no_of_test_doc/df_test[w])
print("calculated tfidf_test")

print("starting do svd")
#  reduce dimension of vector using svd
test_data = svd.transform(tfidf_test)
print("finished")

count:0
count:1000
count:2000
count:3000
count:4000
count:5000
count:6000
count:7000
calculated tfidf_test
starting do svd
finished


In [0]:
del X_test
del df_test
del tfidf_test
print("cleared memory")


cleared memory


In [0]:

print("starting to test")
f = open(test_label_path)
reader = csv.reader(f,delimiter=" ")
tlabel={}
i=0;
for rows in reader:
    tlabel[i]=int(rows[0])-1
    i+=1
    

starting to test


In [0]:
number_of_words=5000

Nk=[0]*number_of_class;
for i in range(0,num_of_doc):
    Nk[int(label[i])]=Nk[int(label[i])]+1;

mean=numpy.zeros(shape=(number_of_class,number_of_words))
for c in range(0,number_of_class):
    for w in range(0,number_of_words):
        mean[c][w]=train_data[c][w]/Nk[c]
print("calculated mean")

# calculate covariance matrix        
sigma=numpy.zeros(shape=(number_of_words,number_of_words))
for d in range(0,num_of_doc):
    temp=train_data[d]-mean[int(label[d])]
    sigma=sigma+numpy.outer(temp,temp)

sigma=(1/num_of_doc)*sigma
print("calculated sigma")

sigma_inv=numpy.linalg.pinv(sigma)
print("calculated sigma_inv")


calculated mean
calculated sigma
calculated sigma_inv


In [0]:
PrC=[0]*number_of_class
for c in range(0,number_of_class):
    PrC[c]=Nk[c]/num_of_doc

beta=[0]*number_of_class
# beta calcuation
for c in range(0,number_of_class):
    beta[c]=0.5*(mean[c].T.dot(sigma_inv).dot(mean[c]))+math.log(PrC[c])
print("calculated beta")

calculated beta


In [0]:
def getClass(x):
    global beta,mean,sigma_inv
    
#     select arg max
    max_c=0
    max_val=mean[0].T.dot(sigma_inv).dot(x)
    for c in range(1,number_of_class):
        t=mean[c].T.dot(sigma_inv).dot(x)
        if max_val<t:
            max_val=t
            max_c=c
    return max_c;

In [0]:
correct=0
incorrect=0
for i in range(0,no_of_test_doc):
    l=getClass(test_data[i])
    if l==tlabel[i]:
        correct+=1
    else:
        incorrect+=1

print("final result: c: "+str(correct)+"  e: "+str(incorrect))
print("Accuracy:"+str(correct/(correct+incorrect)))